# Segment Surface of Hippocampal Formation

This notebook transforms the 3D segmentation images into 3D meshed surfaces and stores them into .ply files.

The meshes in the .ply files can be opened with:
- the `vscode-3d-preview` extension of VSCode
- [MeshLab](https://www.meshlab.net/)
- [Blender](https://www.blender.org/download/), specifically with its [Stop-motion-OBJ](https://github.com/neverhood311/Stop-motion-OBJ) plugin to visualize time-series of .ply files.

**Note:** Add a plugin to blender following [these instructions](https://docs.blender.org/manual/en/latest/editors/preferences/addons.html#rd-party-add-ons). 

## Set-up

In [3]:
import os
import subprocess

gitroot_path = subprocess.check_output(
    ["git", "rev-parse", "--show-toplevel"], universal_newlines=True
)
os.chdir(gitroot_path[:-1])
print("Working directory: ", os.getcwd())

import warnings

warnings.filterwarnings("ignore")

import sys

sys_dir = os.path.dirname(os.getcwd())
sys.path.append(sys_dir)
print("Directory added to path: ", sys_dir)

Working directory:  /home/adele/code/my28brains
Directory added to path:  /home/adele/code


## Imports

In [4]:
import glob

import my28brains.io_mesh as io_mesh
import nibabel
import numpy as np
import skimage
import trimesh

INFO: Using numpy backend


## Writing Directories

In [5]:
# create "days" strings for each folder in 28 and me data folder.
days = []
for i_day in np.arange(1, 10):
    days.append("Day0" + str(i_day))
for i_day in np.arange(10, 61):
    days.append("Day" + str(i_day))

print(days)

['Day01', 'Day02', 'Day03', 'Day04', 'Day05', 'Day06', 'Day07', 'Day08', 'Day09', 'Day10', 'Day11', 'Day12', 'Day13', 'Day14', 'Day15', 'Day16', 'Day17', 'Day18', 'Day19', 'Day20', 'Day21', 'Day22', 'Day23', 'Day24', 'Day25', 'Day26', 'Day27', 'Day28', 'Day29', 'Day30', 'Day31', 'Day32', 'Day33', 'Day34', 'Day35', 'Day36', 'Day37', 'Day38', 'Day39', 'Day40', 'Day41', 'Day42', 'Day43', 'Day44', 'Day45', 'Day46', 'Day47', 'Day48', 'Day49', 'Day50', 'Day51', 'Day52', 'Day53', 'Day54', 'Day55', 'Day56', 'Day57', 'Day58', 'Day59', 'Day60']


In [6]:
# IMG_DIR = "/home/data/28andme/"
MESHES_DIR = os.path.join(os.getcwd(), "data", "meshes")

IMG_DIRS = []
for day in days:
    IMG_DIRS.append(
        os.path.join("/home/adele/code/my28brains/data/28andMeOC_correct", day)
    )
print("IMG_DIRS: ", IMG_DIRS)
print("MESHES_DIR: ", MESHES_DIR)

IMG_DIRS:  ['/home/adele/code/my28brains/data/28andMeOC_correct/Day01', '/home/adele/code/my28brains/data/28andMeOC_correct/Day02', '/home/adele/code/my28brains/data/28andMeOC_correct/Day03', '/home/adele/code/my28brains/data/28andMeOC_correct/Day04', '/home/adele/code/my28brains/data/28andMeOC_correct/Day05', '/home/adele/code/my28brains/data/28andMeOC_correct/Day06', '/home/adele/code/my28brains/data/28andMeOC_correct/Day07', '/home/adele/code/my28brains/data/28andMeOC_correct/Day08', '/home/adele/code/my28brains/data/28andMeOC_correct/Day09', '/home/adele/code/my28brains/data/28andMeOC_correct/Day10', '/home/adele/code/my28brains/data/28andMeOC_correct/Day11', '/home/adele/code/my28brains/data/28andMeOC_correct/Day12', '/home/adele/code/my28brains/data/28andMeOC_correct/Day13', '/home/adele/code/my28brains/data/28andMeOC_correct/Day14', '/home/adele/code/my28brains/data/28andMeOC_correct/Day15', '/home/adele/code/my28brains/data/28andMeOC_correct/Day16', '/home/adele/code/my28brains

## Functions to Extract Meshes

In [7]:
def _extract_mesh(img_fdata, structure_id):
    """Extract one surface mesh from the fdata of a segmented image."""
    if structure_id == -1:
        img_mask = img_fdata != 0
    else:
        img_mask = img_fdata == structure_id
    meshing_result = skimage.measure.marching_cubes(img_mask)
    mesh = trimesh.Trimesh(vertices=meshing_result[0], faces=meshing_result[1])
    return mesh


def extract_mesh(nii_path, structure_id=-1):
    """Extract surface mesh(es) from a structure in the hippocampal formation.

    The nii_path should contain the segmentation of the hippocampal formation,
    with the following structures (given from their corresponding integer labels):
    1: CA1
    2: CA23
    3: Dentate Gyrus (DG)
    4: Entorhinal Cortex (ERC)
    5: Perirhinal Cortex (PRC)
    6: Parahippocampal Cortex (PHC)
    7: Subiculum (SUB)
    8: Anterior Hipp
    9: Posterior Hipp

    Note that the label 0 denotes the background.

    Parameters
    ----------
    nii_path : str
        Path of the .nii.gz image with the segmented structures.
    structure_id : int or list
        Index or list of indices of the anatomical structure(s) to
        mesh from the hippocampal formation.
        Possible indices are either -1 (entire structure) or any of the
        labels of the segmentation.

    Return
    ------
    mesh : trimesh.Trimesh or list of trimesh.Trimesh's
        Surface mesh (or list of meshes) of the anatomical structure.
    """
    print(f"Loading image from {nii_path}...")
    img = nibabel.load(nii_path)
    img_fdata = img.get_fdata()

    if isinstance(structure_id, int):
        return _extract_mesh(img_fdata, structure_id)

    meshes = []
    for one_structure_id in structure_id:
        one_mesh = _extract_mesh(img_fdata, one_structure_id)
        meshes.append(one_mesh)
    return meshes

## Left Hemisphere: Extract and Write Meshes

In [8]:
# structure_ids = list(range(1,4))
# # structure_ids.append(-1)
# print(structure_ids)

[1, 2, 3]


In [10]:
hemisphere = "left"
structure_ids = list(range(1,10))
structure_ids.append(-1)

for structure_id in structure_ids:
    string_bases = []
    for i_day, IMG_DIR in enumerate(IMG_DIRS):
        for file_name in os.listdir(IMG_DIR):
            if file_name.startswith(hemisphere) and file_name.endswith(".nii.gz"):
                string_bases.append(os.path.join(IMG_DIR, file_name))
                break

    print(len(string_bases))

    for i_path, nii_path in enumerate(string_bases):
        mesh = extract_mesh(nii_path=nii_path, structure_id=structure_id)
        day = i_path + 1

        ply_path = os.path.join(
            # MESHES_DIR, f"{hemisphere}_structure_{structure_id}_{sub}_{ses}.ply"
            MESHES_DIR,
            f"{hemisphere}_structure_{structure_id}_day{day:02}.ply",
        )
        io.write_trimesh_to_ply(mesh=mesh, ply_path=ply_path)

60
Loading image from /home/adele/code/my28brains/data/28andMeOC_correct/Day01/left_lfseg_corr_usegray_CT_LQ.nii.gz...
Writing mesh at /home/adele/code/my28brains/data/meshes/left_structure_1_day01.ply...
Loading image from /home/adele/code/my28brains/data/28andMeOC_correct/Day02/left_lfseg_corr_usegray_CT_LQ.nii.gz...
Writing mesh at /home/adele/code/my28brains/data/meshes/left_structure_1_day02.ply...
Loading image from /home/adele/code/my28brains/data/28andMeOC_correct/Day03/left_lfseg_corr_usegray_CT_LQ.nii.gz...
Writing mesh at /home/adele/code/my28brains/data/meshes/left_structure_1_day03.ply...
Loading image from /home/adele/code/my28brains/data/28andMeOC_correct/Day04/left_lfseg_corr_usegray_CT_LQ.nii.gz...
Writing mesh at /home/adele/code/my28brains/data/meshes/left_structure_1_day04.ply...
Loading image from /home/adele/code/my28brains/data/28andMeOC_correct/Day05/left_lfseg_corr_usegray_CT_LQ.nii.gz...
Writing mesh at /home/adele/code/my28brains/data/meshes/left_structure_1_

## Right Hemisphere: Segment and Write Meshes

In [15]:
hemisphere = "right"
structure_ids = list(range(1,10))
structure_ids.append(-1)

for structure_id in structure_ids:
    string_bases = []
    for i_day, IMG_DIR in enumerate(IMG_DIRS):
        for file_name in os.listdir(IMG_DIR):
            if file_name.startswith(hemisphere) and file_name.endswith(".nii.gz"):
                string_bases.append(os.path.join(IMG_DIR, file_name))
                break

    print(len(string_bases))

    for i_path, nii_path in enumerate(string_bases):
        mesh = extract_mesh(nii_path=nii_path, structure_id=structure_id)
        day = i_path + 1

        ply_path = os.path.join(
            # MESHES_DIR, f"{hemisphere}_structure_{structure_id}_{sub}_{ses}.ply"
            MESHES_DIR,
            f"{hemisphere}_structure_{structure_id}_day{day:02}.ply",
        )
        io.write_trimesh_to_ply(mesh=mesh, ply_path=ply_path)

58
Loading image from /home/adele/code/my28brains/data/28andMeOC_correct/Day01/right_lfseg_corr_usegray_CT_LQ.nii.gz...
Writing mesh at /home/adele/code/my28brains/data/meshes/right_structure_1_day01.ply...
Loading image from /home/adele/code/my28brains/data/28andMeOC_correct/Day02/right_lfseg_corr_usegray_CT_LQ.nii.gz...
Writing mesh at /home/adele/code/my28brains/data/meshes/right_structure_1_day02.ply...
Loading image from /home/adele/code/my28brains/data/28andMeOC_correct/Day03/right_lfseg_corr_usegray_CT_LQPost.nii.gz...
Writing mesh at /home/adele/code/my28brains/data/meshes/right_structure_1_day03.ply...
Loading image from /home/adele/code/my28brains/data/28andMeOC_correct/Day04/right_lfseg_corr_usegray_CT_LQ.nii.gz...
Writing mesh at /home/adele/code/my28brains/data/meshes/right_structure_1_day04.ply...
Loading image from /home/adele/code/my28brains/data/28andMeOC_correct/Day05/right_lfseg_corr_usegray_CT_adjustedLI-tail.nii.gz...
Writing mesh at /home/adele/code/my28brains/dat